<table style="width: 100%;">
    <tr>
        <td><a href="https://ieb-chile.cl/en/" target="_blank"><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/IEB.png" style="height: 100px;"></a</td>
        <td></td>
        <td><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/Biodata.png" style="height: 100px;"></td>
    </tr>
</table>

# Merge Catalogue of Life and Catalogue of Vascular Plant

The final steps consist in merging the two generated archives:
1. The Catalogue of Life DwC-A with the species filtered according to GBIF, without vascular plants.
2. The Catalogue of Vascular Plants of Chile curated by the University of Concepción.

In [1]:
from dwca import DarwinCoreArchive
from eml.types import ResponsibleParty, I18nString
from xml_common.utils import Language

In [2]:
col = DarwinCoreArchive.from_file("data/chilean_col_var.zip")
col

/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: https://terms.catalogueoflife.org/notho not in expected namespace for http://rs.tdwg.org/dwc/terms/Taxon class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:75: UserWarning: http://rs.gbif.org/terms/1.0/Distribution not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:75: UserWarning: http://rs.gbif.org/terms/1.0/VernacularName not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "


Reading file Taxon.tsv:   0%|          | 0/106202 [00:00<?, ?entry/s]

Reading file Distribution.tsv:   0%|          | 0/32403 [00:00<?, ?entry/s]

Reading file VernacularName.tsv:   0%|          | 0/27936 [00:00<?, ?entry/s]

Reading file SpeciesProfile.tsv:   0%|          | 0/22326 [00:00<?, ?entry/s]

<Darwin Core Archive (col-clb-299029 [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 106202])>

Different Language URI. Fixed to match pydwca especifications:
http://purl.org/dc/elements/1.1/language -> http://purl.org/dc/terms/language

In [3]:
vern_data = col.extensions[1].pandas
vern_data

Converting to pandas:   0%|          | 0/27936 [00:00<?, ?entry/s]

,taxonID,language,vernacularName
0,8TNXX,eng,Dujardin's slime sponge
1,8TNXX,deu,Dujardins Gallertschwamm
2,8TNXX,deu,Gallertschwamm
3,8TNXX,eng,soft horny sponge
4,8TNXX,nld,weke sponsje
...,...,...,...
27931,C992K,spa,quimeras
27932,C992K,eng,ratfishes
27933,C992K,eng,shortnose chimaeras
27934,C992K,eng,shortnosed chimaeras


In [4]:
from dwca.classes import DataFile, DataFileType
from dwca.terms import Field, TaxonID, DWCLanguage, VernacularName as DWCVernacularName

In [5]:
vern_data["language"] = vern_data["language"].apply(DWCLanguage(1).format)
vern_data

,taxonID,language,vernacularName
0,8TNXX,Language.ENG,Dujardin's slime sponge
1,8TNXX,Language.DEU,Dujardins Gallertschwamm
2,8TNXX,Language.DEU,Gallertschwamm
3,8TNXX,Language.ENG,soft horny sponge
4,8TNXX,Language.NLD,weke sponsje
...,...,...,...
27931,C992K,Language.SPA,quimeras
27932,C992K,Language.ENG,ratfishes
27933,C992K,Language.ENG,shortnose chimaeras
27934,C992K,Language.ENG,shortnosed chimaeras


Erase two letters code conflict languages

In [6]:
conflicted_languages = [
    Language.DNJ, Language.UND, Language.MEY,
    Language.CMN, Language.SCO, Language.HAW,
    Language.ROM, Language.PAP, Language.NIU,
    Language.SRN, Language.BCN, Language.VIF,
    Language.TKL, Language.NLG, Language.BIK,
    Language.TSI, Language.CAL, Language.PAU,
    Language.ACE, Language.PAG, Language.ILO,
    Language.WAR, Language.PAM, Language.CEB,
    Language.HAI, Language.FON, Language.FAN,
    Language.TVL, Language.HIL, Language.MEN,
    Language.RAP, Language.KOK, Language.KOS,
    Language.SUS
]

In [7]:
vern_data[vern_data["language"].isin(conflicted_languages)]

,taxonID,language,vernacularName
250,JTCQ,Language.DNJ,vandmand
277,LLTX,Language.DNJ,melongople
412,3V2RM,Language.DNJ,variabel slimbændel
462,GG8W,Language.UND,Bhebguez
468,GG8W,Language.UND,Garibguez
...,...,...,...
26685,8NKQ9,Language.DNJ,Kanttæger
26867,4XNCR,Language.UND,Green planthopper
26868,4XNCR,Language.UND,Green planthopper
26869,4XNCR,Language.UND,Torpedo bug


In [8]:
vern_data = vern_data[~vern_data["language"].isin(conflicted_languages)].copy().reset_index(drop=True)
vern_data

,taxonID,language,vernacularName
0,8TNXX,Language.ENG,Dujardin's slime sponge
1,8TNXX,Language.DEU,Dujardins Gallertschwamm
2,8TNXX,Language.DEU,Gallertschwamm
3,8TNXX,Language.ENG,soft horny sponge
4,8TNXX,Language.NLD,weke sponsje
...,...,...,...
23446,C992K,Language.SPA,quimeras
23447,C992K,Language.ENG,ratfishes
23448,C992K,Language.ENG,shortnose chimaeras
23449,C992K,Language.ENG,shortnosed chimaeras


In [9]:
from typing import List

class VernacularName(DataFile):
    URI = "http://rs.gbif.org/terms/1.0/VernacularName"
    def __init__(self, _id: int, files: str, fields: List[Field]):
        super().__init__(
            _id, files, fields, DataFileType.EXTENSION,
            "utf-8", "\n", "\t",
            "", 1
        )
        return

vern_col = VernacularName(0, "VernacularName.tsv", [TaxonID(0), DWCLanguage(1, two_letter_coding=True), DWCVernacularName(2)])
vern_col.as_pandas()
vern_col.pandas = vern_data

print(vern_col)

Converting to pandas: 0entry [00:00, ?entry/s]

Extension:
	class: http://rs.gbif.org/terms/1.0/VernacularName
	filename: VernacularName.tsv
	content: 23451 entries


In [10]:
col.extensions[1] = vern_col

In [11]:
vascular_plant = DarwinCoreArchive.from_file("data/vascular_plant_catalogue.zip")
vascular_plant

/home/ubuntu/pydwca/src/dwca/base/darwincore_archive.py:75: UserWarning: http://rs.gbif.org/terms/1.0/VernacularName not in expected namespace. Some functionalities may not be available.
  warn(f"{element.get('rowType')} not in expected namespace. "
/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: http://rs.tdwg.org/dwc/terms/occurrenceStatus not in expected namespace for http://rs.tdwg.org/dwc/terms/Event class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: http://rs.tdwg.org/dwc/terms/locality not in expected namespace for http://rs.tdwg.org/dwc/terms/Event class. Some functionalities may not be available.
  warn(f"{element.get('term')} not in expected namespace for "
/home/ubuntu/pydwca/src/dwca/classes/data_file.py:285: UserWarning: http://rs.tdwg.org/dwc/terms/country not in expected namespace for http://rs.tdwg.org/dwc/terms/Event clas

Reading file taxon.tsv:   0%|          | 0/20359 [00:00<?, ?entry/s]

Reading file vernacular.tsv:   0%|          | 0/1924 [00:00<?, ?entry/s]

Reading file distribution.tsv:   0%|          | 0/22824 [00:00<?, ?entry/s]

Reading file reference.tsv:   0%|          | 0/3 [00:00<?, ?entry/s]

<Darwin Core Archive (catalogoplantas.udec.cl [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 20359])>

In [12]:
merged_archive = DarwinCoreArchive.merge(col, vascular_plant)
merged_archive

Converting to pandas:   0%|          | 0/106202 [00:00<?, ?entry/s]

Converting to pandas:   0%|          | 0/20359 [00:00<?, ?entry/s]

/home/ubuntu/pydwca/src/dwca/classes/data_file.py:668: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged.pandas = pd.concat([data, data_file.pandas], axis=0).reset_index(drop=True)


Converting to pandas:   0%|          | 0/1924 [00:00<?, ?entry/s]

<Darwin Core Archive (None [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 126561])>

In [13]:
merged_archive.__id__ = "https://portalbiodiversidad.cl"
merged_archive.generate_eml("eml.xml")
merged_archive.metadata.initialize_resource(
    "Portal of Biodiversity Chile",
    ResponsibleParty(
        organization_name=I18nString("Instituto de Ecología y Biodiversidad - Chile", Language.SPA),
    ),
    contact=[ResponsibleParty(
        position_name=I18nString("Informatic Researcher", Language.ENG),
    )]
)

In [14]:
merged_archive.core.pandas

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
0,9SC75,9JMSF,4QVLW,,--08KihF3V8CBd-0AiQUc,170394,synonym,species,Hypnum spiniforme,Hedw.,...,Bryophyta,Bryopsida,Rhizogoniales,Calomniaceae,Pyrrhobryum,None,NaT,None,None,None
1,9JNLJ,9M7TP,9JJTK,,--0uNw9zMZNm0dcI01yk50,170394,synonym,genus,Swartzia,Brid.,...,Bryophyta,Bryopsida,Distichiales,Distichiaceae,Swartzia,None,NaT,None,None,None
2,7THWB,7PKCR,7TJ35,t6tkMX5AIody4pYGk7B400,--2DJkMYBP9L9OCFNDT7W,1130,synonym,species,Turritella (Peyrotia) patagonica,"G. B. Sowerby I, 1846",...,Mollusca,Gastropoda,,Turritellidae,Turritella,None,NaT,None,None,None
3,57SM6,CCBK3,7CKGH,,--4Vz50zmsPL4KJA32FyN1,1010,synonym,species,Trachyrhynchus villegai,"Pequeño, 1971",...,Chordata,Actinopterygii,Gadiformes,Macrouridae,Trachyrincus,None,NaT,None,None,None
4,75TWC,CCY6B,42LSH,,--AUbvKR8zsU9_0vsolF6,2073,synonym,species,Patellaria trisepta,(Nägeli) Müll. Arg.,...,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Micarea,None,NaT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126556,https://catalogoplantas.udec.cl/taxa/20084,https://catalogoplantas.udec.cl/taxa/6796,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/20084,catalogoplantas.udec.cl,synonym,species,Zephyra violiflora,(Bertero ex Colla) Ravenna,...,Magnoliophyta,Liliopsida,Asparagales,Tecophilaeaceae,Tecophilaea,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126557,https://catalogoplantas.udec.cl/taxa/20085,https://catalogoplantas.udec.cl/taxa/4423,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/20085,catalogoplantas.udec.cl,synonym,species,Zoellnerallium andinum,(Poepp.) Crosa,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Latace,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126558,https://catalogoplantas.udec.cl/taxa/20086,https://catalogoplantas.udec.cl/taxa/662,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/20086,catalogoplantas.udec.cl,synonym,species,Zuccagnia angulata,Hook. & Arn.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Erythrostemon,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126559,https://catalogoplantas.udec.cl/taxa/20087,https://catalogoplantas.udec.cl/taxa/767,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/20087,catalogoplantas.udec.cl,synonym,species,Zuccagnia eremophila,Phil.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Hoffmannseggia,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


## Resolve conflict

The Kingdom _Plantae_ is in both dataset, therefore, to resolve this, we are going to leave the one from **Catalogue of Life** and referencing from the new division values:

In [15]:
data = merged_archive.core.pandas
data

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
0,9SC75,9JMSF,4QVLW,,--08KihF3V8CBd-0AiQUc,170394,synonym,species,Hypnum spiniforme,Hedw.,...,Bryophyta,Bryopsida,Rhizogoniales,Calomniaceae,Pyrrhobryum,None,NaT,None,None,None
1,9JNLJ,9M7TP,9JJTK,,--0uNw9zMZNm0dcI01yk50,170394,synonym,genus,Swartzia,Brid.,...,Bryophyta,Bryopsida,Distichiales,Distichiaceae,Swartzia,None,NaT,None,None,None
2,7THWB,7PKCR,7TJ35,t6tkMX5AIody4pYGk7B400,--2DJkMYBP9L9OCFNDT7W,1130,synonym,species,Turritella (Peyrotia) patagonica,"G. B. Sowerby I, 1846",...,Mollusca,Gastropoda,,Turritellidae,Turritella,None,NaT,None,None,None
3,57SM6,CCBK3,7CKGH,,--4Vz50zmsPL4KJA32FyN1,1010,synonym,species,Trachyrhynchus villegai,"Pequeño, 1971",...,Chordata,Actinopterygii,Gadiformes,Macrouridae,Trachyrincus,None,NaT,None,None,None
4,75TWC,CCY6B,42LSH,,--AUbvKR8zsU9_0vsolF6,2073,synonym,species,Patellaria trisepta,(Nägeli) Müll. Arg.,...,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Micarea,None,NaT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126556,https://catalogoplantas.udec.cl/taxa/20084,https://catalogoplantas.udec.cl/taxa/6796,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/20084,catalogoplantas.udec.cl,synonym,species,Zephyra violiflora,(Bertero ex Colla) Ravenna,...,Magnoliophyta,Liliopsida,Asparagales,Tecophilaeaceae,Tecophilaea,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126557,https://catalogoplantas.udec.cl/taxa/20085,https://catalogoplantas.udec.cl/taxa/4423,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/20085,catalogoplantas.udec.cl,synonym,species,Zoellnerallium andinum,(Poepp.) Crosa,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Latace,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126558,https://catalogoplantas.udec.cl/taxa/20086,https://catalogoplantas.udec.cl/taxa/662,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/20086,catalogoplantas.udec.cl,synonym,species,Zuccagnia angulata,Hook. & Arn.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Erythrostemon,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126559,https://catalogoplantas.udec.cl/taxa/20087,https://catalogoplantas.udec.cl/taxa/767,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/20087,catalogoplantas.udec.cl,synonym,species,Zuccagnia eremophila,Phil.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Hoffmannseggia,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


In [16]:
data[data["taxonRank"] == "kingdom"]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
1770,N,5T6MX,N,,061950e4-9782-4d1a-9c87-dcf375788e6b,,accepted,kingdom,Animalia,,...,,,,,,None,NaT,None,None,None
3257,F,5T6MX,F,,0wJCTw-fhjeF9tJGV0I-u1,2073,accepted,kingdom,Fungi,,...,,,,,,None,NaT,None,None,None
12190,C,5T6MX,C,,63e291b2-9018-4cdb-a599-ed6404d06ab4,,accepted,kingdom,Chromista,,...,,,,,,None,NaT,None,None,None
73149,P,5T6MX,P,,fc995438-b561-4616-a896-f479fca6e3d4,,accepted,kingdom,Plantae,,...,,,,,,None,NaT,None,None,None
106202,https://catalogoplantas.udec.cl/taxa/1,,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/1,catalogoplantas.udec.cl,accepted,kingdom,Plantae,,...,,,,,,[],2022-07-08 15:52:05+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


Changing the taxa that reference _Plantae_

In [17]:
data[(data["parentNameUsageID"] == "P") | (data["parentNameUsageID"] == "https://catalogoplantas.udec.cl/taxa/1")]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
97312,9THX2,P,9THX2,,uNtDOBSCgOc41OxiMCOft,170394,accepted,subkingdom,Bryobiotina,Trevis.,...,,,,,,None,NaT,None,None,None
106203,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,catalogoplantas.udec.cl,accepted,division,Magnoliophyta,,...,Magnoliophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106204,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,catalogoplantas.udec.cl,accepted,division,Pinophyta,,...,Pinophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106205,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,catalogoplantas.udec.cl,accepted,division,Pteridophyta,,...,Pteridophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


In [18]:
data[data["parentNameUsageID"] == "https://catalogoplantas.udec.cl/taxa/1"]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
106203,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,catalogoplantas.udec.cl,accepted,division,Magnoliophyta,,...,Magnoliophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106204,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,catalogoplantas.udec.cl,accepted,division,Pinophyta,,...,Pinophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106205,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,catalogoplantas.udec.cl,accepted,division,Pteridophyta,,...,Pteridophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


In [19]:
data.loc[
    data[data["parentNameUsageID"] == "https://catalogoplantas.udec.cl/taxa/1"].index,
    "parentNameUsageID"
] = "P"
data[(data["parentNameUsageID"] == "P") | (data["parentNameUsageID"] == "https://catalogoplantas.udec.cl/taxa/1")]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
97312,9THX2,P,9THX2,,uNtDOBSCgOc41OxiMCOft,170394,accepted,subkingdom,Bryobiotina,Trevis.,...,,,,,,None,NaT,None,None,None
106203,https://catalogoplantas.udec.cl/taxa/2,P,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,catalogoplantas.udec.cl,accepted,division,Magnoliophyta,,...,Magnoliophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106204,https://catalogoplantas.udec.cl/taxa/3,P,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,catalogoplantas.udec.cl,accepted,division,Pinophyta,,...,Pinophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
106205,https://catalogoplantas.udec.cl/taxa/4,P,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,catalogoplantas.udec.cl,accepted,division,Pteridophyta,,...,Pteridophyta,,,,,[Plantae],2022-07-08 15:53:17+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


Drop new _Plantae_ and re-index

In [20]:
data.drop(
    data[data["taxonID"] == "https://catalogoplantas.udec.cl/taxa/1"].index,
    inplace=True
)
data.reset_index(drop=True, inplace=True)
data

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
0,9SC75,9JMSF,4QVLW,,--08KihF3V8CBd-0AiQUc,170394,synonym,species,Hypnum spiniforme,Hedw.,...,Bryophyta,Bryopsida,Rhizogoniales,Calomniaceae,Pyrrhobryum,None,NaT,None,None,None
1,9JNLJ,9M7TP,9JJTK,,--0uNw9zMZNm0dcI01yk50,170394,synonym,genus,Swartzia,Brid.,...,Bryophyta,Bryopsida,Distichiales,Distichiaceae,Swartzia,None,NaT,None,None,None
2,7THWB,7PKCR,7TJ35,t6tkMX5AIody4pYGk7B400,--2DJkMYBP9L9OCFNDT7W,1130,synonym,species,Turritella (Peyrotia) patagonica,"G. B. Sowerby I, 1846",...,Mollusca,Gastropoda,,Turritellidae,Turritella,None,NaT,None,None,None
3,57SM6,CCBK3,7CKGH,,--4Vz50zmsPL4KJA32FyN1,1010,synonym,species,Trachyrhynchus villegai,"Pequeño, 1971",...,Chordata,Actinopterygii,Gadiformes,Macrouridae,Trachyrincus,None,NaT,None,None,None
4,75TWC,CCY6B,42LSH,,--AUbvKR8zsU9_0vsolF6,2073,synonym,species,Patellaria trisepta,(Nägeli) Müll. Arg.,...,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Micarea,None,NaT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126555,https://catalogoplantas.udec.cl/taxa/20084,https://catalogoplantas.udec.cl/taxa/6796,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/20084,catalogoplantas.udec.cl,synonym,species,Zephyra violiflora,(Bertero ex Colla) Ravenna,...,Magnoliophyta,Liliopsida,Asparagales,Tecophilaeaceae,Tecophilaea,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126556,https://catalogoplantas.udec.cl/taxa/20085,https://catalogoplantas.udec.cl/taxa/4423,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/20085,catalogoplantas.udec.cl,synonym,species,Zoellnerallium andinum,(Poepp.) Crosa,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Latace,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126557,https://catalogoplantas.udec.cl/taxa/20086,https://catalogoplantas.udec.cl/taxa/662,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/20086,catalogoplantas.udec.cl,synonym,species,Zuccagnia angulata,Hook. & Arn.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Erythrostemon,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126558,https://catalogoplantas.udec.cl/taxa/20087,https://catalogoplantas.udec.cl/taxa/767,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/20087,catalogoplantas.udec.cl,synonym,species,Zuccagnia eremophila,Phil.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Hoffmannseggia,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


In [21]:
data[data["taxonRank"] == "kingdom"]

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
1770,N,5T6MX,N,,061950e4-9782-4d1a-9c87-dcf375788e6b,,accepted,kingdom,Animalia,,...,,,,,,None,NaT,None,None,None
3257,F,5T6MX,F,,0wJCTw-fhjeF9tJGV0I-u1,2073,accepted,kingdom,Fungi,,...,,,,,,None,NaT,None,None,None
12190,C,5T6MX,C,,63e291b2-9018-4cdb-a599-ed6404d06ab4,,accepted,kingdom,Chromista,,...,,,,,,None,NaT,None,None,None
73149,P,5T6MX,P,,fc995438-b561-4616-a896-f479fca6e3d4,,accepted,kingdom,Plantae,,...,,,,,,None,NaT,None,None,None


## Saving new DwC-A

In [22]:
merged_archive.core.pandas = data
merged_archive.to_file("data/portal-dwca.zip")

Writing data http://rs.tdwg.org/dwc/terms/Taxon:   0%|          | 0/126560 [00:00<?, ?line/s]

Writing data http://rs.gbif.org/terms/1.0/Distribution:   0%|          | 0/32403 [00:00<?, ?line/s]

Writing data http://rs.gbif.org/terms/1.0/VernacularName:   0%|          | 0/25375 [00:00<?, ?line/s]

Writing data http://rs.gbif.org/terms/1.0/SpeciesProfile:   0%|          | 0/22326 [00:00<?, ?line/s]

Writing data http://rs.tdwg.org/dwc/terms/Event:   0%|          | 0/22824 [00:00<?, ?line/s]

Writing data http://rs.gbif.org/terms/1.0/Reference:   0%|          | 0/3 [00:00<?, ?line/s]

# Counting species

Now we are exploring the data by counting all terminal taxa (species, subespecies, varieties and forms).

In [23]:
import pandas as pd

In [24]:
taxon_df = merged_archive.core.pandas
taxon_df

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
0,9SC75,9JMSF,4QVLW,,--08KihF3V8CBd-0AiQUc,170394,synonym,species,Hypnum spiniforme,Hedw.,...,Bryophyta,Bryopsida,Rhizogoniales,Calomniaceae,Pyrrhobryum,None,NaT,None,None,None
1,9JNLJ,9M7TP,9JJTK,,--0uNw9zMZNm0dcI01yk50,170394,synonym,genus,Swartzia,Brid.,...,Bryophyta,Bryopsida,Distichiales,Distichiaceae,Swartzia,None,NaT,None,None,None
2,7THWB,7PKCR,7TJ35,t6tkMX5AIody4pYGk7B400,--2DJkMYBP9L9OCFNDT7W,1130,synonym,species,Turritella (Peyrotia) patagonica,"G. B. Sowerby I, 1846",...,Mollusca,Gastropoda,,Turritellidae,Turritella,None,NaT,None,None,None
3,57SM6,CCBK3,7CKGH,,--4Vz50zmsPL4KJA32FyN1,1010,synonym,species,Trachyrhynchus villegai,"Pequeño, 1971",...,Chordata,Actinopterygii,Gadiformes,Macrouridae,Trachyrincus,None,NaT,None,None,None
4,75TWC,CCY6B,42LSH,,--AUbvKR8zsU9_0vsolF6,2073,synonym,species,Patellaria trisepta,(Nägeli) Müll. Arg.,...,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Micarea,None,NaT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126555,https://catalogoplantas.udec.cl/taxa/20084,https://catalogoplantas.udec.cl/taxa/6796,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/6798,https://catalogoplantas.udec.cl/taxa/20084,catalogoplantas.udec.cl,synonym,species,Zephyra violiflora,(Bertero ex Colla) Ravenna,...,Magnoliophyta,Liliopsida,Asparagales,Tecophilaeaceae,Tecophilaea,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126556,https://catalogoplantas.udec.cl/taxa/20085,https://catalogoplantas.udec.cl/taxa/4423,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/4424,https://catalogoplantas.udec.cl/taxa/20085,catalogoplantas.udec.cl,synonym,species,Zoellnerallium andinum,(Poepp.) Crosa,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Latace,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126557,https://catalogoplantas.udec.cl/taxa/20086,https://catalogoplantas.udec.cl/taxa/662,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/3614,https://catalogoplantas.udec.cl/taxa/20086,catalogoplantas.udec.cl,synonym,species,Zuccagnia angulata,Hook. & Arn.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Erythrostemon,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
126558,https://catalogoplantas.udec.cl/taxa/20087,https://catalogoplantas.udec.cl/taxa/767,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/4158,https://catalogoplantas.udec.cl/taxa/20087,catalogoplantas.udec.cl,synonym,species,Zuccagnia eremophila,Phil.,...,Magnoliophyta,Magnoliopsida,Fabales,Fabaceae,Hoffmannseggia,"[Plantae, Magnoliophyta, Magnoliopsida, Fabale...",2024-06-28 15:34:38+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


Checking just terminal taxa

In [25]:
terminals = taxon_df[
    (~taxon_df["taxonID"].isin(taxon_df["parentNameUsageID"])) &
    (taxon_df["taxonomicStatus"] == "accepted") &
    (taxon_df["taxonRank"].isin(["species", "form", "variety", "subspecies", "subform", "subvariety"]))
]
terminals

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,scientificNameAuthorship,...,phylum,class,order,family,genus,higherClassification,modified,institutionID,institutionCode,datasetName
9,Q7Q7,9JHYR,Q7Q7,SLBsFT_5VicoA06nJYUvd,--XyyFPjxBv19KSuuvm0u0,170394,accepted,species,Calyptrochaeta apiculata,(Hook. f. & Wilson) Vitt,...,Bryophyta,Bryopsida,Hookeriales,Daltoniaceae,Calyptrochaeta,None,NaT,None,None,None
30,53MXK,84TZC,53MXK,,-0ozGCMRqn904Gj9hUmGC0,1090,accepted,species,Syllis variegata,"Grube, 1860",...,Annelida,Polychaeta,Phyllodocida,Syllidae,Syllis,None,NaT,None,None,None
40,9CNFP,9CJHX,9CNFP,,-1IxlBwKwr9H9Swg4q1MF,2144,accepted,species,Acanthephyra purpurea,"A. Milne-Edwards, 1881",...,Arthropoda,Malacostraca,Decapoda,Acanthephyridae,Acanthephyra,None,NaT,None,None,None
45,74BJP,659L,74BJP,,-1UbHGLQYJcIAtdh1pox51,2144,accepted,species,Notiothauma reedi,"MacLachlan, 1877",...,Arthropoda,Insecta,Mecoptera,Eomeropidae,Notiothauma,None,NaT,None,None,None
46,7488M,8MT2D,7488M,ScIHUn-YSZA-EhTM0fusM,-1ZVpb_gMIcfESnwj_yvx1,1065,accepted,species,Neofulla spinosa,"(Aubert, 1960)",...,Arthropoda,Insecta,Plecoptera,Notonemouridae,Neofulla,None,NaT,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113562,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/7255,catalogoplantas.udec.cl,accepted,species,Zephyranthes sarae,J.M. Watson & A.R. Flores,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-07-23 21:32:42+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
113563,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/7256,catalogoplantas.udec.cl,accepted,species,Zephyranthes splendens,(Renjifo) Nic. García,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-07-23 21:32:42+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
113564,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/7257,catalogoplantas.udec.cl,accepted,species,Zephyranthes tenuiflora,(Phil.) Nic.García,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-07-23 21:32:42+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile
113565,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/7258,catalogoplantas.udec.cl,accepted,species,Zephyranthes tubispatha,(L'Hér.) Herb.,...,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,"[Plantae, Magnoliophyta, Liliopsida, Asparagal...",2024-07-23 21:32:42+00:00,https://udec.cl,UDEC,Catálogo de las plantas vasculares de Chile


In [26]:
terminals.columns

Index(['taxonID', 'parentNameUsageID', 'acceptedNameUsageID',
       'originalNameUsageID', 'scientificNameID', 'datasetID',
       'taxonomicStatus', 'taxonRank', 'scientificName',
       'scientificNameAuthorship', 'notho', 'genericName',
       'infragenericEpithet', 'specificEpithet', 'infraspecificEpithet',
       'cultivarEpithet', 'nameAccordingTo', 'namePublishedIn',
       'nomenclaturalCode', 'nomenclaturalStatus', 'taxonRemarks',
       'references', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus',
       'higherClassification', 'modified', 'institutionID', 'institutionCode',
       'datasetName'],
      dtype='object')

In [27]:
terminals.groupby("kingdom").size()

kingdom
Animalia     10402
Chromista      642
Fungi         2730
Plantae       6887
dtype: int64

In [28]:
terminals.groupby("phylum").size()

phylum
Acanthocephala              2
Annelida                  527
Anthocerotophyta           19
Arthropoda               5046
Ascomycota               1800
Basidiomycota             857
Bigyra                      1
Brachiopoda                21
Bryophyta                 828
Bryozoa                    92
Cercozoa                    1
Chaetognatha                1
Chordata                 1967
Chytridiomycota            17
Ciliophora                 45
Cnidaria                  357
Ctenophora                  4
Echinodermata             318
Entomophthoromycota         3
Entorrhizomycota            1
Foraminifera              513
Glomeromycota               5
Hemichordata                4
Kinorhyncha                 4
Loricifera                  1
Magnoliophyta            5378
Marchantiophyta           473
Miozoa                      7
Mollusca                 1476
Mucoromycota               44
Nematoda                  189
Nemertea                   41
Neocallimastigomycota       1
Och

<table>
    <tr>
        <td colspan="3" style="text-align: center;"><p>BIODATA - <a href="https://ieb-chile.cl/en/" target="_blank">Institute of Ecology and Biodiversity</a> © 2025</p></td>
    </tr>
</table>